Gather necessary pre-requisites:

In [15]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
from nilearn import plotting, datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nb
import os, re
from itertools import zip_longest

200302-07:59:50,918 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


In [2]:
# define useful functions
# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
# add suptitle via plt.suptitle
        
# check size of items and delete them when above a certain threshold
from sys import getsizeof # function is not fully indicative -> look for better option
def check_itemsize(items):
    for i in items:
        # return size of object in bytes
        print("size of item", i, "is:", getsizeof(i))
        if getsizeof(i) > 10000: # think about dynamic coding for this
            del i
            print("deleted, object exceeded set threshold.")
        else:
            print("retained, object is sufficiently small.")

Get files

In [62]:
# collect all spmT images [contrasts 1-4: dep>con, con>dep, male>female, young>old]
# specify relevant directories
VBM_dir = "/VBM/"
norm_dirs = [os.path.join(VBM_dir, f"{normdir}") for normdir in os.listdir(VBM_dir) 
                          if re.match(r'DARTEL_norm2mni.*', normdir)]
MReg_dirs = [os.path.join(normdir, regdir) for normdir in norm_dirs 
             for regdir in os.listdir(normdir) if re.match(r'SPM_M.*', regdir)]
MReg_ests = [os.path.join(regdir, estdir) for regdir in MReg_dirs 
             for estdir in os.listdir(regdir) if re.match(r'est.*', estdir)]
Contrast_dirs = [os.path.join(estdir, contdir) for estdir in MReg_ests 
              for contdir in os.listdir(estdir) if re.match(r'est.*Cont.*', contdir)]

# define the split GM and WM templates
DARTEL_templates = [VBM_dir + f"DARTEL_template/grouptemplate_6_{i}.nii" for i in ["GM", "WM"]]

# get spmT images
spmT_img = sorted([os.path.join(contdir, img) for contdir in Contrast_dirs for img in os.listdir(contdir) 
                   if re.match(r'(^spmT.*)', img)])

# get directories of FDR thresholded images
FDR_dirs = [os.path.join(contdir, f"FDR_spmT{contrast_index}") 
            for contdir in Contrast_dirs for contrast_index in range(1, (len(spmT_img)//2)+1)]

# get the spmT images after FDR thresholding (show only surviving clusters)
spmT_img_FDR = [os.path.join(fdrdir, img) for fdrdir in FDR_dirs for img in os.listdir(fdrdir) 
                if re.match(r'(^spmT.*\d{4}_thr.nii)', img)]

# define contrast names in order of creation (MReg model)
con_names = ["dep>con", "con>dep", "male>female", "young>old"]

Plot VBM results

In [63]:
# get surf_mesh background for 3D plotting
#fsaverage = datasets.fetch_surf_fsaverage()

# prep image
# flexible type error -> look at nb object and access datatype
#malefemale_nb = nb.load(spmT_img_FDR[2])
# get func data, change datatype from nan to float32
#malefemale_fdata = malefemale_nb.dataobj[:].astype(np.float32)
# save back to nifti, using Identity matrix as affine transformation
#malefemale_fdata_img = nb.Nifti1Image(malefemale_fdata, np.eye(4))
#malefemale_fdata_img.to_filename(os.path.join(FDR_dirs[2], "spmT3.nii.gz"))

#stat_img_surf_3D = plotting.plot_surf_stat_map(stat_map=os.path.join(FDR_dirs[2], "spmT3.nii.gz"), title=con_names[2], 
#                                       surf_mesh=fsaverage['pial_left'], bg_map=fsaverage['sulc_left'], 
#                                       hemi='left', view='medial', bg_on_data=True)
#stat_img_surf3D = plotting.view_img_on_surf(spmT_img_FDR[2], cmap="black_red", colorbar=True, colorbar_height=0.5, 
#                                            surf_mesh='fsaverage', title=con_names[2])

# plot results only for relevant contrast male>female
# for GM
stat_img_3D_GM = plotting.view_img(spmT_img_FDR[2], threshold=0, cut_coords=[-42, -16, 52], title=con_names[2])
stat_img_3D_GM.save_as_html(os.path.join(FDR_dirs[2], "spmT3_img_GM.html"))
stat_map_zslice10_GM = plotting.plot_stat_map(spmT_img_FDR[2], threshold=0, display_mode="z", cut_coords=10, title=con_names[2])
stat_map_zslice10_GM.savefig(os.path.join(FDR_dirs[2], "spmT3_statmap_Zslice10_GM.png"))
stat_map_zslice10_ontemplate_GM = plotting.plot_stat_map(spmT_img_FDR[2], threshold=0, display_mode="z", cut_coords=10, 
                                                         bg_img=DARTEL_templates[0], title=con_names[2])
stat_map_zslice10_ontemplate_GM.savefig(os.path.join(FDR_dirs[2], "spmT3_statmap_Zslice10_ontemplate_GM.png"))

# for WM
stat_img_3D_GM = plotting.view_img(spmT_img_FDR[6], threshold=0, cut_coords=[-42, -16, 52], title=con_names[2])
stat_img_3D_GM.save_as_html(os.path.join(FDR_dirs[6], "spmT3_img_WM.html"))
stat_map_zslice10_GM = plotting.plot_stat_map(spmT_img_FDR[6], threshold=0, display_mode="z", cut_coords=10, title=con_names[2])
stat_map_zslice10_GM.savefig(os.path.join(FDR_dirs[6], "spmT3_statmap_Zslice10_WM.png"))
stat_map_zslice10_ontemplate_GM = plotting.plot_stat_map(spmT_img_FDR[6], threshold=0, display_mode="z", cut_coords=10, 
                                                         bg_img=DARTEL_templates[1], title=con_names[2])
stat_map_zslice10_ontemplate_GM.savefig(os.path.join(FDR_dirs[6], "spmT3_statmap_Zslice10_ontemplate_WM.png"))

Testing Zone

In [61]:
print(FDR_dirs[6])

/VBM/DARTEL_norm2mni_c2/SPM_MultiReg_WM/estimate_MReg/estimate_Contrats/FDR_spmT3


In [ ]:
# make presentation figure showing middle slice of one struct and one fMRI
# activate in-line magic
%matplotlib inline

# get the relevant volume out of the 4D nifti
fmri1=nimg.load_img(FMRI[0])
fmri1_middle=nimg.index_img(fmri1, fmri1.shape[3] // 2)

#plot everything
bettercolours=plot_epi(fmri1_middle, cut_coords=(10,10,40),
          display_mode='ortho', cmap="CMRmap", draw_cross=False, annotate=False)
normal=plot_epi(fmri1_middle, cut_coords=(10,10,40),
          display_mode='ortho', cmap="gray", draw_cross=False, annotate=False)

# safe files for presentation

bettercolours.savefig("/fMRI/fMRI_bettercolours.png")
normal.savefig("/fMRI/fMRI_normal.png")
struct.savefig("/fMRI/structMRI_normal.png")